In [42]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV,cross_val_score
import matplotlib.pyplot as plt
from sklearn import tree
import seaborn as sns
plt.rcParams['figure.figsize'] = (10,5)
plt.rcParams['figure.dpi'] = 1000
sns.set_theme(style='darkgrid', palette='rainbow')
import warnings
warnings.filterwarnings('ignore')

In [27]:
df = pd.read_csv(r'C:\Users\shree\Desktop\subject\Data_Science\DS_Assignments\Datasets\Company_Data.csv')
df

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...
395,12.57,138,108,17,203,128,Good,33,14,Yes,Yes
396,6.14,139,23,3,37,120,Medium,55,11,No,Yes
397,7.41,162,26,12,368,159,Medium,40,18,Yes,Yes
398,5.94,100,79,7,284,95,Bad,50,12,Yes,Yes


In [15]:
df.shape

(400, 11)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Sales        400 non-null    float64
 1   CompPrice    400 non-null    int64  
 2   Income       400 non-null    int64  
 3   Advertising  400 non-null    int64  
 4   Population   400 non-null    int64  
 5   Price        400 non-null    int64  
 6   ShelveLoc    400 non-null    object 
 7   Age          400 non-null    int64  
 8   Education    400 non-null    int64  
 9   Urban        400 non-null    object 
 10  US           400 non-null    object 
dtypes: float64(1), int64(7), object(3)
memory usage: 34.5+ KB


In [17]:
df.describe()

,Sales,CompPrice,Income,Advertising,Population,Price,Age,Education
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,7.496325,124.975000,68.657500,6.635000,264.840000,115.795000,53.322500,13.900000
std,2.824115,15.334512,27.986037,6.650364,147.376436,23.676664,16.200297,2.620528
min,0.000000,77.000000,21.000000,0.000000,10.000000,24.000000,25.000000,10.000000
25%,5.390000,115.000000,42.750000,0.000000,139.000000,100.000000,39.750000,12.000000
50%,7.490000,125.000000,69.000000,5.000000,272.000000,117.000000,54.500000,14.000000
75%,9.320000,135.000000,91.000000,12.000000,398.500000,131.000000,66.000000,16.000000
max,16.270000,175.000000,120.000000,29.000000,509.000000,191.000000,80.000000,18.000000


In [28]:
def outlier_cap(df,column):
    for index, row in df.iterrows():
        h = df[column].quantile(0.75)
        l = df[column].quantile(0.25)
        IQR = h - l
        upper_threshold = h + 1.5*IQR
        lower_threshold = l - 1.5*IQR
        if row[column] > upper_threshold:
            df.at[index, column] = upper_threshold
        elif row[column] < lower_threshold:
            
            df.at[index, column] = lower_threshold

In [29]:
col = ['Sales', 'CompPrice', 'Income', 'Advertising', 'Population', 'Price',
     'Age', 'Education']

In [30]:
for i in col:
    outlier_cap(df,i)

In [31]:
df

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120.0,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83.0,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80.0,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97.0,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128.0,Bad,38,13,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...
395,12.57,138,108,17,203,128.0,Good,33,14,Yes,Yes
396,6.14,139,23,3,37,120.0,Medium,55,11,No,Yes
397,7.41,162,26,12,368,159.0,Medium,40,18,Yes,Yes
398,5.94,100,79,7,284,95.0,Bad,50,12,Yes,Yes


In [22]:
df['Sales'].mean()

7.492650000000001

In [32]:
df['Sale'] = np.where(df['Sales'] >= df['Sales'].mean() ,'high' , 'low')
df

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US,Sale
0,9.50,138,73,11,276,120.0,Bad,42,17,Yes,Yes,high
1,11.22,111,48,16,260,83.0,Good,65,10,Yes,Yes,high
2,10.06,113,35,10,269,80.0,Medium,59,12,Yes,Yes,high
3,7.40,117,100,4,466,97.0,Medium,55,14,Yes,Yes,low
4,4.15,141,64,3,340,128.0,Bad,38,13,Yes,No,low
...,...,...,...,...,...,...,...,...,...,...,...,...
395,12.57,138,108,17,203,128.0,Good,33,14,Yes,Yes,high
396,6.14,139,23,3,37,120.0,Medium,55,11,No,Yes,low
397,7.41,162,26,12,368,159.0,Medium,40,18,Yes,Yes,low
398,5.94,100,79,7,284,95.0,Bad,50,12,Yes,Yes,low


In [33]:
df['ShelveLoc'].replace({'Bad': 1, "Medium": 2,"Good":3},inplace = True)

In [35]:
df['Sale'].replace({'low': 0, "high": 1},inplace = True)

In [36]:
df

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US,Sale
0,9.50,138,73,11,276,120.0,1,42,17,Yes,Yes,1
1,11.22,111,48,16,260,83.0,3,65,10,Yes,Yes,1
2,10.06,113,35,10,269,80.0,2,59,12,Yes,Yes,1
3,7.40,117,100,4,466,97.0,2,55,14,Yes,Yes,0
4,4.15,141,64,3,340,128.0,1,38,13,Yes,No,0
...,...,...,...,...,...,...,...,...,...,...,...,...
395,12.57,138,108,17,203,128.0,3,33,14,Yes,Yes,1
396,6.14,139,23,3,37,120.0,2,55,11,No,Yes,0
397,7.41,162,26,12,368,159.0,2,40,18,Yes,Yes,0
398,5.94,100,79,7,284,95.0,1,50,12,Yes,Yes,0


In [37]:
df_encoded = pd.get_dummies(df,drop_first = True)

In [38]:
df_encoded

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Sale,Urban_Yes,US_Yes
0,9.50,138,73,11,276,120.0,1,42,17,1,1,1
1,11.22,111,48,16,260,83.0,3,65,10,1,1,1
2,10.06,113,35,10,269,80.0,2,59,12,1,1,1
3,7.40,117,100,4,466,97.0,2,55,14,0,1,1
4,4.15,141,64,3,340,128.0,1,38,13,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
395,12.57,138,108,17,203,128.0,3,33,14,1,1,1
396,6.14,139,23,3,37,120.0,2,55,11,0,0,1
397,7.41,162,26,12,368,159.0,2,40,18,0,1,1
398,5.94,100,79,7,284,95.0,1,50,12,0,1,1


In [39]:
x = df_encoded.drop(columns = ['Sales','Sale'])
x

,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban_Yes,US_Yes
0,138,73,11,276,120.0,1,42,17,1,1
1,111,48,16,260,83.0,3,65,10,1,1
2,113,35,10,269,80.0,2,59,12,1,1
3,117,100,4,466,97.0,2,55,14,1,1
4,141,64,3,340,128.0,1,38,13,1,0
...,...,...,...,...,...,...,...,...,...,...
395,138,108,17,203,128.0,3,33,14,1,1
396,139,23,3,37,120.0,2,55,11,0,1
397,162,26,12,368,159.0,2,40,18,1,1
398,100,79,7,284,95.0,1,50,12,1,1


In [47]:
sc = StandardScaler()
x_scaled = sc.fit_transform(x)

In [49]:
pd.DataFrame(x_scaled)

,0,1,2,3,4,5,6,7,8,9
0,0.858149,0.155361,0.657177,0.075819,0.179646,-1.446917,-0.699782,1.184449,0.646869,0.741881
1,-0.920057,-0.739060,1.409957,-0.032882,-1.413859,1.528747,0.721723,-1.490113,0.646869,0.741881
2,-0.788338,-1.204159,0.506621,0.028262,-1.543062,0.040915,0.350895,-0.725953,0.646869,0.741881
3,-0.524900,1.121336,-0.396715,1.366649,-0.810911,0.040915,0.103677,0.038208,0.646869,0.741881
4,1.055727,-0.166631,-0.547271,0.510625,0.524188,-1.446917,-0.947000,-0.343872,0.646869,-1.347925
...,...,...,...,...,...,...,...,...,...,...
395,0.858149,1.407551,1.560513,-0.420131,0.524188,1.528747,-1.256023,0.038208,0.646869,0.741881
396,0.924008,-1.633482,-0.547271,-1.547909,0.179646,0.040915,0.103677,-1.108033,-1.545908,0.741881
397,2.438776,-1.526151,0.807733,0.700853,1.859287,0.040915,-0.823391,1.566529,0.646869,0.741881
398,-1.644511,0.370022,0.054953,0.130170,-0.897046,-1.446917,-0.205346,-0.725953,0.646869,0.741881


In [50]:
y = df_encoded['Sale']

In [51]:
y

0      1
1      1
2      1
3      0
4      0
      ..
395    1
396    0
397    0
398    0
399    1
Name: Sale, Length: 400, dtype: int64

In [52]:
xtrain,xtest,ytrain,ytest = train_test_split(x_scaled,y,test_size=0.2, random_state=1)

In [53]:
rfc = RandomForestClassifier()
rfc.fit(xtrain,ytrain)
yp = rfc.predict(xtest)


In [54]:
yp

array([0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0], dtype=int64)

In [55]:
print(classification_report(ytest,yp))

              precision    recall  f1-score   support

           0       0.75      0.82      0.79        40
           1       0.81      0.72      0.76        40

    accuracy                           0.78        80
   macro avg       0.78      0.77      0.77        80
weighted avg       0.78      0.78      0.77        80



In [56]:
np.mean(cross_val_score(RandomForestClassifier(),xtrain,ytrain,cv = 10 ,scoring = 'accuracy'))

0.809375

In [57]:
len(rfc.estimators_)

100

In [58]:
params = {
    'n_estimators' : [20,60,100,120],
    'criterion' : ["gini", "entropy", "log_loss"],
    'max_depth' : [2,3,4,5,6,7,8],
    'max_features': [0.2,0.4,0.6,0.8,1],
 'max_samples': [0.25,0.5,0.75,1,1.25]
}

In [59]:
grid = GridSearchCV(estimator=RandomForestClassifier(), param_grid=params,cv = 5,
                   verbose=2,n_jobs = -1)

In [60]:
grid.fit(xtrain,ytrain)

Fitting 5 folds for each of 2100 candidates, totalling 10500 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [2, 3, 4, 5, 6, 7, 8],
                         'max_features': [0.2, 0.4, 0.6, 0.8, 1],
                         'max_samples': [0.25, 0.5, 0.75, 1, 1.25],
                         'n_estimators': [20, 60, 100, 120]},
             verbose=2)

In [61]:
grid.best_params_

{'criterion': 'gini',
 'max_depth': 7,
 'max_features': 0.2,
 'max_samples': 0.25,
 'n_estimators': 100}

In [62]:
criteria = grid.best_params_['criterion']
max_depth = grid.best_params_['max_depth']
max_features = grid.best_params_['max_features']
max_samples = grid.best_params_['max_samples']
n_estimators = grid.best_params_['n_estimators']

In [63]:
print(criteria,max_depth,max_features,max_samples,n_estimators)

gini 7 0.2 0.25 100


In [64]:
dtr = RandomForestClassifier(criterion=criteria,max_depth=max_depth, 
                             max_features= max_features,max_samples= max_samples,n_estimators =n_estimators)

In [65]:
dtr.fit(xtrain,ytrain)
ypred  = dtr.predict(xtest)

In [66]:
dtr.score(xtrain,ytrain),dtr.score(xtest,ytest)

(0.928125, 0.8)

In [67]:
print(classification_report(ypred , ytest))

              precision    recall  f1-score   support

           0       0.85      0.77      0.81        44
           1       0.75      0.83      0.79        36

    accuracy                           0.80        80
   macro avg       0.80      0.80      0.80        80
weighted avg       0.81      0.80      0.80        80



In [68]:
np.mean(cross_val_score(dtr,xtrain,ytrain,cv = 10 ,scoring = 'accuracy'))

0.790625

## Fraud Dataset

In [69]:
fraud = pd.read_csv(r'C:\Users\shree\Desktop\subject\Data_Science\DS_Assignments\Datasets\Fraud_check.csv')
fraud

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO
...,...,...,...,...,...,...
595,YES,Divorced,76340,39492,7,YES
596,YES,Divorced,69967,55369,2,YES
597,NO,Divorced,47334,154058,0,YES
598,YES,Married,98592,180083,17,NO


In [70]:
fraud.shape

(600, 6)

In [71]:
fraud.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Undergrad        600 non-null    object
 1   Marital.Status   600 non-null    object
 2   Taxable.Income   600 non-null    int64 
 3   City.Population  600 non-null    int64 
 4   Work.Experience  600 non-null    int64 
 5   Urban            600 non-null    object
dtypes: int64(3), object(3)
memory usage: 28.2+ KB


In [72]:
fraud.duplicated().sum()

0

In [73]:
fraud.describe()

,Taxable.Income,City.Population,Work.Experience
count,600.000000,600.000000,600.000000
mean,55208.375000,108747.368333,15.558333
std,26204.827597,49850.075134,8.842147
min,10003.000000,25779.000000,0.000000
25%,32871.500000,66966.750000,8.000000
50%,55074.500000,106493.500000,15.000000
75%,78611.750000,150114.250000,24.000000
max,99619.000000,199778.000000,30.000000


In [74]:
fraud['Defaulter'] = np.where(fraud['Taxable.Income'] <= 30000 , 'Risky' , 'good')
fraud

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban,Defaulter
0,NO,Single,68833,50047,10,YES,good
1,YES,Divorced,33700,134075,18,YES,good
2,NO,Married,36925,160205,30,YES,good
3,YES,Single,50190,193264,15,YES,good
4,NO,Married,81002,27533,28,NO,good
...,...,...,...,...,...,...,...
595,YES,Divorced,76340,39492,7,YES,good
596,YES,Divorced,69967,55369,2,YES,good
597,NO,Divorced,47334,154058,0,YES,good
598,YES,Married,98592,180083,17,NO,good


In [75]:
fraud[fraud['Defaulter'] == 'Risky']

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban,Defaulter
10,NO,Single,29732,102602,19,YES,Risky
12,NO,Divorced,11794,148033,14,YES,Risky
16,YES,Divorced,24987,34551,29,NO,Risky
19,NO,Single,10987,126953,30,YES,Risky
21,YES,Divorced,14310,29106,7,YES,Risky
...,...,...,...,...,...,...,...
544,NO,Single,29916,133877,21,YES,Risky
557,YES,Married,12810,157421,10,NO,Risky
567,YES,Single,16316,129912,30,NO,Risky
575,NO,Divorced,10735,95975,13,YES,Risky


In [76]:
fraud['Defaulter'].replace({'good': 0, "Risky": 1},inplace = True)

In [77]:
fraud_encoded = pd.get_dummies(fraud,drop_first = True)

In [78]:
fraud_encoded

,Taxable.Income,City.Population,Work.Experience,Defaulter,Undergrad_YES,Marital.Status_Married,Marital.Status_Single,Urban_YES
0,68833,50047,10,0,0,0,1,1
1,33700,134075,18,0,1,0,0,1
2,36925,160205,30,0,0,1,0,1
3,50190,193264,15,0,1,0,1,1
4,81002,27533,28,0,0,1,0,0
...,...,...,...,...,...,...,...,...
595,76340,39492,7,0,1,0,0,1
596,69967,55369,2,0,1,0,0,1
597,47334,154058,0,0,0,0,0,1
598,98592,180083,17,0,1,1,0,0


In [79]:
x1 = fraud_encoded.drop(columns = ['Defaulter'])
x1

,Taxable.Income,City.Population,Work.Experience,Undergrad_YES,Marital.Status_Married,Marital.Status_Single,Urban_YES
0,68833,50047,10,0,0,1,1
1,33700,134075,18,1,0,0,1
2,36925,160205,30,0,1,0,1
3,50190,193264,15,1,0,1,1
4,81002,27533,28,0,1,0,0
...,...,...,...,...,...,...,...
595,76340,39492,7,1,0,0,1
596,69967,55369,2,1,0,0,1
597,47334,154058,0,0,0,0,1
598,98592,180083,17,1,1,0,0


In [80]:
sc = StandardScaler()
x1_scaled = sc.fit_transform(x1) 
x1_scaled

array([[ 0.52036187, -1.17852072, -0.62914254, ..., -0.6912543 ,
         1.32852435,  0.99335541],
       [-0.82146395,  0.50850002,  0.27636996, ..., -0.6912543 ,
        -0.75271484,  0.99335541],
       [-0.69829233,  1.03310911,  1.63463872, ...,  1.4466456 ,
        -0.75271484,  0.99335541],
       ...,
       [-0.30074402,  0.90969648, -1.76103318, ..., -0.6912543 ,
        -0.75271484,  0.99335541],
       [ 1.65693986,  1.43219749,  0.1631809 , ...,  1.4466456 ,
        -0.75271484, -1.00668904],
       [ 1.57776629,  0.9915901 ,  0.04999184, ..., -0.6912543 ,
        -0.75271484, -1.00668904]])

In [81]:
y1 = fraud_encoded['Defaulter']
y1

0      0
1      0
2      0
3      0
4      0
      ..
595    0
596    0
597    0
598    0
599    0
Name: Defaulter, Length: 600, dtype: int64

In [82]:
xtrain_f,xtest_f,ytrain_f,ytest_f = train_test_split(x1_scaled,y1,test_size=0.2, random_state=1)

In [83]:
dtr = RandomForestClassifier()

dtr.fit(xtrain_f,ytrain_f)
yp1 = dtr.predict(xtest_f)
yp1

array([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0], dtype=int64)

In [84]:
dtr.score(xtrain_f,ytrain_f), dtr.score(xtest_f,ytest_f)

(1.0, 1.0)

In [85]:
np.mean(cross_val_score(RandomForestClassifier(),xtrain_f,ytrain_f,cv = 10 ,scoring = 'accuracy'))

0.9979166666666666

In [86]:
fraud.columns

Index(['Undergrad', 'Marital.Status', 'Taxable.Income', 'City.Population',
       'Work.Experience', 'Urban', 'Defaulter'],
      dtype='object')

In [87]:
params = {
    'criterion' : ["gini", "entropy", "log_loss"],
    'max_depth' : [2,3,4,5,6,7,8],
    'max_features': [1,2,3,4,5],
    'max_samples': [0.25,0.5,1,1.25,1.5,2],
    'n_estimators':[20,60,100,120]
}

In [88]:
grid = GridSearchCV(estimator=RandomForestClassifier(), param_grid=params,cv = 5,
                   verbose=2,n_jobs = -1)

In [89]:
grid.fit(xtrain_f,ytrain_f)

Fitting 5 folds for each of 2520 candidates, totalling 12600 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [2, 3, 4, 5, 6, 7, 8],
                         'max_features': [1, 2, 3, 4, 5],
                         'max_samples': [0.25, 0.5, 1, 1.25, 1.5, 2],
                         'n_estimators': [20, 60, 100, 120]},
             verbose=2)

In [90]:
criteria_f = grid.best_params_['criterion']
max_depth_f = grid.best_params_['max_depth']
max_features_f = grid.best_params_['max_features']
max_samples_f = grid.best_params_['max_samples']
n_estimators_f = grid.best_params_['n_estimators']

In [91]:
print(criteria_f,max_depth_f,max_features_f,max_samples_f,n_estimators_f)

gini 2 4 0.25 100


In [92]:
dtr = RandomForestClassifier(criterion=criteria_f,max_depth=max_depth_f,max_features = max_features_f, 
                             n_estimators = n_estimators_f,max_samples= max_samples_f)

In [93]:
dtr.fit(xtrain_f,ytrain_f)
ypred_f = dtr.predict(xtest_f)

In [94]:
dtr.score(xtrain_f,ytrain_f),dtr.score(xtest_f,ytest_f)

(0.9979166666666667, 1.0)

In [95]:
np.mean(cross_val_score(dtr,xtrain_f,ytrain_f,cv = 10 ,scoring = 'accuracy'))

1.0

In [96]:
print(classification_report(ypred_f , ytest_f))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        97
           1       1.00      1.00      1.00        23

    accuracy                           1.00       120
   macro avg       1.00      1.00      1.00       120
weighted avg       1.00      1.00      1.00       120

